# ValoML - Automated Scouting Report Generator

## Project Overview

This notebook documents the **ValoML pipeline** - an automated scouting report generator for Valorant esports. The system fetches match data from GRID's esports data API, analyzes team and player performance, and generates actionable insights for coaches.

### Core Question
> **"How do we win against this team?"**

---

## Pipeline Architecture

```
┌─────────────────┐     ┌──────────────────┐     ┌─────────────────┐
│   GRID API      │────▶│  Data Ingestion  │────▶│    Analysis     │
│  (GraphQL/REST) │     │   (FastAPI)      │     │ (Team/Player)   │
└─────────────────┘     └──────────────────┘     └────────┬────────┘
                                                          │
                                                          ▼
┌─────────────────┐     ┌──────────────────┐     ┌─────────────────┐
│   PDF Report    │◀────│   LLM Writer     │◀────│    Insights     │
│   (Markdown)    │     │   (OpenAI)       │     │   Generator     │
└─────────────────┘     └──────────────────┘     └─────────────────┘
```

---

## Phase 0: Install Dependencies

Run this cell first to install required packages.

In [3]:
# Install required dependencies (use %pip for VS Code Jupyter)
%pip install python-dotenv httpx --quiet
print("✅ Dependencies installed!")

Note: you may need to restart the kernel to use updated packages.
✅ Dependencies installed!



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


---

## Phase 1: Environment Setup

Set up the environment and import our modules.

In [4]:
# Add backend to path for imports
import sys
sys.path.insert(0, './backend')

# Import configuration
from config import get_settings

settings = get_settings()
print(f"App: {settings.app_name}")
print(f"GRID Central Data URL: {settings.grid_central_data_url}")
print(f"API Key configured: {'Yes' if settings.grid_api_key else 'No'}")

App: ValoML Scouting Report Generator
GRID Central Data URL: https://api-op.grid.gg/central-data/graphql
API Key configured: Yes


---

## Phase 2: Data Ingestion

The data ingestion layer connects to GRID's esports API to fetch:
- **Teams**: Search and retrieve team information
- **Series**: Match history for a team
- **Games**: Individual map details within a series

### 2.1 GRID GraphQL Client

We use a GraphQL client to query GRID's Central Data API.

In [5]:
from clients.grid_client import GridClient
import asyncio

# Initialize the GRID client
client = GridClient()

# Test connection - fetch available game titles
async def test_connection():
    titles = await client.get_all_titles()
    return titles

# For Jupyter, use nest_asyncio or run in new loop
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    pass

titles = asyncio.get_event_loop().run_until_complete(test_connection())
print("Available titles in GRID:")
for title in titles[:5]:
    print(f"  - {title.get('name')} (ID: {title.get('id')})")

Available titles in GRID:


### 2.2 Team Search

Search for a team by name to get their ID and metadata.

In [6]:
# Search for a team
TEAM_NAME = "Sentinels"  # Change this to analyze different teams

async def search_team(name):
    teams = await client.search_team(name)
    return teams

teams = asyncio.get_event_loop().run_until_complete(search_team(TEAM_NAME))

print(f"Found {len(teams)} team(s) matching '{TEAM_NAME}':")
for team in teams:
    title = team.get('title', {}).get('name', 'Unknown')
    print(f"  - {team.get('name')} (ID: {team.get('id')}, Game: {title})")

# Select the Valorant team
selected_team = None
for t in teams:
    if t.get('title', {}).get('name', '').lower() == 'valorant':
        selected_team = t
        break
if not selected_team and teams:
    selected_team = teams[0]

print(f"\nSelected team: {selected_team.get('name')} (ID: {selected_team.get('id')})")

Found 3 team(s) matching 'Sentinels':
  - Sentinels (ID: 1079, Game: Valorant)
  - Sentinels Cubert Academy (ID: 54522, Game: Valorant)
  - Sentinels (ID: 56389, Game: League of Legends)

Selected team: Sentinels (ID: 1079)


### 2.3 Fetch Match History

Retrieve recent Valorant series (matches) for the selected team.

In [7]:
# Fetch recent series
NUM_MATCHES = 10

async def fetch_series(team_id, limit):
    series = await client.get_valorant_team_series(team_id, limit)
    return series

series_list = asyncio.get_event_loop().run_until_complete(
    fetch_series(selected_team['id'], NUM_MATCHES)
)

print(f"Retrieved {len(series_list)} Valorant series for {selected_team['name']}:")
for i, series in enumerate(series_list[:5], 1):
    date = series.get('startTimeScheduled', 'Unknown')[:10]
    tournament = series.get('tournament', {}).get('name', 'Unknown')
    print(f"  {i}. {date} - {tournament}")

Retrieved 10 Valorant series for Sentinels:
  1. 2025-08-30 - VCT Americas - Stage 2 2025 (Playoffs: Playoffs)
  2. 2025-08-29 - VCT Americas - Stage 2 2025 (Playoffs: Playoffs)
  3. 2025-08-22 - VCT Americas - Stage 2 2025 (Playoffs: Playoffs)
  4. 2025-08-17 - VCT Americas - Stage 2 2025 (Regular Season: Group A)
  5. 2025-08-09 - VCT Americas - Stage 2 2025 (Regular Season: Group A)


### 2.4 Fetch Detailed Series Data

Get detailed information for each series including individual game/map results.

In [8]:
# Fetch detailed series data
async def fetch_series_details(series_id):
    return await client.get_series_details(series_id)

detailed_series = []
for series in series_list:
    try:
        details = asyncio.get_event_loop().run_until_complete(
            fetch_series_details(series['id'])
        )
        if details:
            detailed_series.append(details)
    except Exception as e:
        print(f"Error fetching series {series['id']}: {e}")

print(f"\nFetched detailed data for {len(detailed_series)} series")

# Show sample structure
if detailed_series:
    sample = detailed_series[0]
    print(f"\nSample series structure:")
    print(f"  - ID: {sample.get('id')}")
    print(f"  - Games: {len(sample.get('games', []))}")
    print(f"  - Teams: {[t.get('baseInfo', {}).get('name') for t in sample.get('teams', [])]}")


Fetched detailed data for 0 series


---

## Phase 3: Data Analysis

The analysis layer processes match data to extract meaningful statistics and patterns.

### 3.1 Team Analyzer

Calculates team-level statistics:
- Win rates (series, maps)
- Side preference (Attack vs Defense)
- Strong/weak maps

In [9]:
from analysis.team_analyzer import TeamAnalyzer

# Initialize team analyzer
team_analyzer = TeamAnalyzer(
    team_id=selected_team['id'],
    team_name=selected_team['name']
)

# Add all series data
team_analyzer.add_series_list(detailed_series)

# Get statistics
stats = team_analyzer.get_stats_dict()

print(f"=== Team Statistics: {stats['team_name']} ===")
print(f"\nSeries Record: {stats['series_wins']}W - {stats['series_losses']}L ({stats['series_win_rate']}%)")
print(f"Map Record: {stats['map_wins']}W - {stats['map_losses']}L ({stats['map_win_rate']}%)")
print(f"\nSide Performance:")
print(f"  - Attack: {stats['attack_win_rate']}%")
print(f"  - Defense: {stats['defense_win_rate']}%")
print(f"  - Preference: {team_analyzer.get_side_preference()}")

=== Team Statistics: Sentinels ===

Series Record: 0W - 0L (0.0%)
Map Record: 0W - 0L (0.0%)

Side Performance:
  - Attack: 0.0%
  - Defense: 0.0%
  - Preference: balanced


In [10]:
# Map analysis
print("\n=== Map Performance ===")
print("\nWeak Maps (< 40% win rate):")
for weak in team_analyzer.get_weak_maps():
    print(f"  - {weak['map']}: {weak['win_rate']}% ({weak['games']} games)")

print("\nStrong Maps (> 60% win rate):")
for strong in team_analyzer.get_strong_maps():
    print(f"  - {strong['map']}: {strong['win_rate']}% ({strong['games']} games)")


=== Map Performance ===

Weak Maps (< 40% win rate):

Strong Maps (> 60% win rate):


### 3.2 Player Analyzer

Analyzes individual player performance:
- Agent pool and pick rates
- KDA statistics
- First blood rate
- Star player / weak link identification

In [12]:
from analysis.player_analyzer import PlayerAnalyzer

# Initialize player analyzer
player_analyzer = PlayerAnalyzer(team_id=selected_team['id'])

# Add series data
for series in detailed_series:
    player_analyzer.add_series_data(series)

# Team agent pool
agent_pool = player_analyzer.get_team_agent_pool()

print("=== Team Agent Pool ===")
print(f"Unique agents used: {agent_pool['unique_agents']}")
print(f"\nTop agents:")
for agent in agent_pool['agents'][:5]:
    print(f"  - {agent['agent']}: {agent['pick_rate']}% pick rate, {agent['win_rate']}% win rate")

=== Team Agent Pool ===
Unique agents used: 0

Top agents:


In [13]:
# Individual player stats
print("\n=== Player Statistics ===")
for player in player_analyzer.get_all_player_stats():
    print(f"\n{player['player_name']}:")
    print(f"  KDA: {player['kda']}")
    print(f"  First Blood Rate: {player['first_blood_rate']}%")
    print(f"  Main Agents: {[a['agent'] for a in player['main_agents']]}")


=== Player Statistics ===


### 3.3 Insight Generator

Converts raw statistics into **actionable insights** following the format:

```
Fact (Data) → Consequence → Recommendation
```

In [14]:
from analysis.insight_generator import InsightGenerator

# Generate insights
insight_gen = InsightGenerator(
    team_analyzer=team_analyzer,
    player_analyzer=player_analyzer
)

insights = insight_gen.generate_all_insights()

print("=== Generated Insights ===")
print(f"Total insights: {len(insights)}")

for i, insight in enumerate(insights[:5], 1):
    print(f"\n--- Insight {i} (Priority: {insight['priority']}) ---")
    print(f"FACT: {insight['fact']}")
    print(f"CONSEQUENCE: {insight['consequence']}")
    print(f"RECOMMENDATION: {insight['recommendation']}")

=== Generated Insights ===
Total insights: 1

--- Insight 1 (Priority: 3) ---
FACT: Pool d'agents limité (0 agents uniques)
CONSEQUENCE: L'équipe a des compositions prévisibles
RECOMMENDATION: Préparer des contre-picks spécifiques, forcer des bans stratégiques


In [15]:
# Executive summary and recommendations
print("=== Executive Summary ===")
print(insight_gen.get_executive_summary())

print("\n=== How to Win ===")
for i, rec in enumerate(insight_gen.get_how_to_win(), 1):
    print(f"{i}. {rec}")

=== Executive Summary ===
**Sentinels** - 0 séries analysées

- Win rate: 0% (séries), 0% (maps)
- Style: balanced


=== How to Win ===
1. Préparer des contre-picks spécifiques, forcer des bans stratégiques


---

## Phase 4: Report Generation (Coming Soon)

The final phase will integrate an LLM to:
1. Take the structured insights (JSON)
2. Generate a professional scouting report
3. Export to PDF format

```python
# Placeholder for LLM integration
# from report.llm_writer import ReportWriter
# from report.pdf_generator import PDFGenerator
```

---

## API Endpoints Summary

The FastAPI backend exposes the following endpoints:

| Endpoint | Method | Description |
|----------|--------|-------------|
| `/api/data/teams/search` | GET | Search teams by name |
| `/api/data/teams/{id}/series` | GET | Get team's match history |
| `/api/data/series/{id}` | GET | Get detailed series info |
| `/api/analysis/team` | POST | Full team analysis |
| `/api/analysis/quick/{name}` | GET | Quick analysis |

**Swagger Docs**: http://localhost:8001/docs

---

## Tech Stack

- **Backend**: Python + FastAPI
- **Data**: GRID Esports API (GraphQL + REST)
- **Analysis**: Custom Python modules (no pandas required)
- **LLM**: OpenAI API (Phase 4)
- **PDF**: Markdown → PDF (Phase 4)
- **Frontend**: Next.js (Phase 5)